In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os
os.chdir('gdrive/My Drive/Colab Notebooks')

Mounted at /content/gdrive


In [2]:
%run Network.ipynb
%run Datasets.ipynb
%run SSIM.ipynb

In [3]:
import os
from os import listdir
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
import matplotlib.pyplot as plt
import torchvision.utils as utils
from skimage.measure.simple_metrics import compare_psnr

#from utils import *

In [4]:
# Image 的前置處理

trns = transforms.Compose([transforms.Scale((150,150))])

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        PSNR += compare_psnr(Iclean[i,:,:,:], Img[i,:,:,:], data_range=data_range)
    return (PSNR/Img.shape[0])

In [6]:
# 讀取資料集
print("Loading datasets...")
            
# 運用 dataloader 操作 datasets
datasets_train = PReNet_train_datasets(transform=trns)
loader_train = DataLoader(dataset=datasets_train, batch_size=8, shuffle=True)

# 建立 model
model = PReNet(6)

# Loss function
criterion = SSIM()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-3)
scheduler = MultiStepLR(optimizer, milestones=[30,50,80], gamma=0.2)

Loading datasets...


In [7]:
# 開始訓練 model

step = 0
for epoch in range(10):
    print('epoch = ', epoch)
    # 進度
    #scheduler.step(epoch)
    for param_group in optimizer.param_groups:
        print('learning rate %f' %param_group["lr"])
    
    # 訓練
    for i, (input_train, target_train) in enumerate(loader_train, 0):
        #o = input_train.permute(0,3,1,2)
        #o = o.reshape(1, 3, 100, 200)
        #o = o.squeeze(0)
        #print(o.shape)
        #new_img = transforms.ToPILImage()(o).convert('RGB')
        #trns = transforms.Compose([transforms.Scale((640,480))])
        #new_img = trns(new_img)
        #display(new_img)    
      
        print("img_size = ", input_train.shape)
        model.train()
        model.zero_grad()
        optimizer.zero_grad()
        input_train, target_train = Variable(input_train), Variable(target_train)

        # training's output
        out_train, _ = model(input_train.permute(0, 3, 1, 2))

        target_train = target_train.permute(0, 3, 1, 2)
        pixel_metric = criterion(target_train, out_train)
        print('pixel_metric = ', pixel_metric)
        loss = -pixel_metric
        
        loss.backward()
        optimizer.step()
        
        # 訓練曲線
        model.eval()
        out_train, _ = model(input_train.permute(0, 3, 1, 2))
        out_train = torch.clamp(out_train, 0., 1.)
        psnr_train = batch_PSNR(out_train, target_train, 1.)
        print("[epoch %d][%d/%d] loss: %.4f, pixel_metric: %.4f, PSNR: %.4f" %
                (epoch+1, i+1, len(loader_train), loss.item(), pixel_metric.item(), psnr_train))
        
        step += 1


        # 輸出影像
        model.eval()
        out_train, _ = model(input_train.permute(0, 3, 1, 2))
        out_train = torch.clamp(out_train, 0., 1.)

        #o = out_train.reshape(1, 3, 100, 200)
        #o = x.squeeze(0)
        
        #new_img = transforms.ToPILImage()(o).convert('RGB')

        #trns = transforms.Compose([transforms.Scale((640,480))])
        #new_img = trns(new_img)
        #display(new_img) 

epoch =  0
learning rate 0.001000
img_size =  torch.Size([8, 150, 150, 3])
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
pixel_metric =  tensor(1.9444e-06, grad_fn=<MeanBackward0>)
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.
  


[epoch 1][1/10] loss: -0.0000, pixel_metric: 0.0000, PSNR: -41.2848
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
img_size =  torch.Size([8, 150, 150, 3])
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
pixel_metric =  tensor(2.1031e-06, grad_fn=<MeanBackward0>)
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
[epoch 1][2/10] loss: -0.0000, pixel_metric: 0.0000, PSNR: -41.1816
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
img_size =  torch.Size([8, 150, 150, 3])
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
pixel_metric =  tensor(1.8675e-06, grad_fn=<MeanBackward0>)


KeyboardInterrupt: ignored

In [ ]:
'''
from IPython.display import display

out_train = out_train.reshape(1, 3, 100, 200)
out_train = out_train.squeeze(0)
new_img = transforms.ToPILImage()(out_train).convert('RGB')

trns1 = transforms.Compose([transforms.Resize((640,480))])

#new_img = trns1(new_img)
display(new_img)
'''

In [ ]:
print(img1.size())

img1 = np.array(img1)
img2 = np.array(img2)
img1 = torch.Tensor(img1)
img2 = torch.Tensor(img2)

img3 = img1*img2
print(img3.size())